Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### Logistic regression with gradient descent and L2 regularization

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

# Regularization coefficient l2_lambda
l2_lambda = 0.1

graph = tf.Graph()
with graph.as_default():
    # Set random seed
    tf.set_random_seed(1)
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    # Add a new l2 regularization term - do not regularize the biases
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
            + l2_lambda * tf.nn.l2_loss(weights)
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
%%time

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 324.707764
Training accuracy: 7.2%
Validation accuracy: 8.8%
Loss at step 100: 0.985591
Training accuracy: 82.0%
Validation accuracy: 81.4%
Loss at step 200: 0.973278
Training accuracy: 82.0%
Validation accuracy: 81.5%
Loss at step 300: 0.972507
Training accuracy: 81.8%
Validation accuracy: 81.5%
Loss at step 400: 0.972275
Training accuracy: 81.9%
Validation accuracy: 81.4%
Loss at step 500: 0.972196
Training accuracy: 81.9%
Validation accuracy: 81.5%
Loss at step 600: 0.972168
Training accuracy: 81.9%
Validation accuracy: 81.5%
Loss at step 700: 0.972157
Training accuracy: 81.9%
Validation accuracy: 81.5%
Loss at step 800: 0.972153
Training accuracy: 81.9%
Validation accuracy: 81.5%
Test accuracy: 87.6%
CPU times: user 7.76 s, sys: 1.4 s, total: 9.16 s
Wall time: 11.6 s


#### Logistic regression with stochastic gradient descent and L2 regularization

In [8]:
batch_size = 128

l2_lambda = 0.1

graph = tf.Graph()
with graph.as_default():
    # Set random seed
    tf.set_random_seed(1)
  
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + l2_lambda * tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
%%time

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 324.785278
Minibatch accuracy: 7.8%
Validation accuracy: 8.9%
Minibatch loss at step 500: 1.039692
Minibatch accuracy: 78.1%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 1.273425
Minibatch accuracy: 71.9%
Validation accuracy: 77.5%
Minibatch loss at step 1500: 1.118569
Minibatch accuracy: 74.2%
Validation accuracy: 79.6%
Minibatch loss at step 2000: 1.065609
Minibatch accuracy: 78.9%
Validation accuracy: 78.9%
Minibatch loss at step 2500: 0.929312
Minibatch accuracy: 84.4%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 1.166695
Minibatch accuracy: 71.9%
Validation accuracy: 78.3%
Test accuracy: 84.8%
CPU times: user 6.52 s, sys: 736 ms, total: 7.25 s
Wall time: 4.55 s


#### Single layer neural net with stochastic gradient descent and L2 regularization

In [10]:
num_nodes = 1024
batch_size = 128
l2_lambda = 0.0025

graph = tf.Graph()
with graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    """VARIABLES"""
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    # Do not regularize biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels)) \
            + l2_lambda * tf.nn.l2_loss(weights_1) \
            + l2_lambda * tf.nn.l2_loss(weights_2)
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_2)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_2)

In [11]:
%%time

num_steps = 3001
print('Regularization L2 term is : ' + str(l2_lambda))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare minibatch dict
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

Regularization L2 term is : 0.0025
Initialized
Minibatch loss at step 0: 1102.1270751953125
Minibatch accuracy: 6.2
Validation accuracy: 40.6
Minibatch loss at step 500: 224.88687133789062
Minibatch accuracy: 81.2
Validation accuracy: 80.3
Minibatch loss at step 1000: 64.22856140136719
Minibatch accuracy: 82.0
Validation accuracy: 83.3
Minibatch loss at step 1500: 18.545513153076172
Minibatch accuracy: 82.0
Validation accuracy: 86.4
Minibatch loss at step 2000: 5.650308609008789
Minibatch accuracy: 89.1
Validation accuracy: 86.6
Minibatch loss at step 2500: 1.9737658500671387
Minibatch accuracy: 89.8
Validation accuracy: 86.7
Minibatch loss at step 3000: 1.1785483360290527
Minibatch accuracy: 81.2
Validation accuracy: 87.4
Test accuracy: 92.9
CPU times: user 16.2 s, sys: 2.39 s, total: 18.6 s
Wall time: 20.2 s


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [12]:
num_nodes = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    """VARIABLES"""
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    # Regularization has been removed
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_2)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_2)

In [13]:
%%time

num_steps = 3001
num_batches = 5

print('Number of batches is: ' + str(num_batches))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = step%num_batches
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare minibatch dict
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

Number of batches is: 5
Initialized
Minibatch loss at step 0: 314.0140075683594
Minibatch accuracy: 6.2
Validation accuracy: 40.6
Minibatch loss at step 500: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 68.2
Minibatch loss at step 1000: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 68.2
Minibatch loss at step 1500: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 68.2
Minibatch loss at step 2000: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 68.2
Minibatch loss at step 2500: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 68.2
Minibatch loss at step 3000: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 68.2
Test accuracy: 74.1
CPU times: user 14 s, sys: 2.21 s, total: 16.2 s
Wall time: 16.1 s


The number of batches was limited to 5 and the weights were not regularized.

Because the neural network 'memorized' and overfit the batches, the classifier did not generalize well to the validation or test dataset.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [14]:
num_nodes = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    """VARIABLES"""
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    
    # Dropout regularization for half of the nodes
    drop_nodes = tf.nn.dropout(relu_layer, 0.5)
    
    logits_2 = tf.matmul(drop_nodes, weights_2) + biases_2
    
    # Regularization has been removed
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_2)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_2)

In [15]:
%%time

num_steps = 3001
num_batches = 5

print('Number of batches is: ' + str(num_batches))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = step%num_batches
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare minibatch dict
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

Number of batches is: 5
Initialized
Minibatch loss at step 0: 468.157958984375
Minibatch accuracy: 10.9
Validation accuracy: 31.1
Minibatch loss at step 500: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 70.7
Minibatch loss at step 1000: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 71.4
Minibatch loss at step 1500: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 71.7
Minibatch loss at step 2000: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 71.9
Minibatch loss at step 2500: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 71.6
Minibatch loss at step 3000: 0.0
Minibatch accuracy: 100.0
Validation accuracy: 71.7
Test accuracy: 78.3
CPU times: user 15.8 s, sys: 2.11 s, total: 17.9 s
Wall time: 19.1 s


Introducing dropout regularization has marginally improved the performance of the previously over-fit neural net with few batches.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Had to adjust parameters as loss function went to `nan` as illustrated [here](http://stackoverflow.com/questions/36565430/adding-multiple-layers-to-tensorflow-causes-loss-function-to-become-nan)

2 layer neural net with L2 regularization

In [16]:
nodes_layer_1 = 1024
nodes_layer_2 = 512
batch_size = 128
l2_lambda = 0.0025
alpha = 0.0025

nn_graph = tf.Graph()
with nn_graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    """VARIABLES"""
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, nodes_layer_1]))
    biases_1  = tf.Variable(tf.zeros([nodes_layer_1]))
    
    weights_2 = tf.Variable(tf.truncated_normal([nodes_layer_1, nodes_layer_2]))
    biases_2  = tf.Variable(tf.zeros([nodes_layer_2]))
    
    weights_3 = tf.Variable(tf.truncated_normal([nodes_layer_2, num_labels]))
    biases_3  = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
    
    
    # Do not regularize biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_3, labels=tf_train_labels)) \
            + l2_lambda * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(alpha).minimize(loss)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_3)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    valid_prediction = tf.nn.softmax(logits_3)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    test_prediction = tf.nn.softmax(logits_3)

In [17]:
%%time

num_steps = 3001

print('Regularization L2 term is : ' + str(l2_lambda))

with tf.Session(graph=nn_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare minibatch dict
        minibatch_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=minibatch_dict)
        
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

Regularization L2 term is : 0.0025
Initialized
Minibatch loss at step 0: 5380.81689453125
Minibatch accuracy: 7.8
Validation accuracy: 17.0
Minibatch loss at step 500: 1498.0167236328125
Minibatch accuracy: 82.0
Validation accuracy: 79.1
Minibatch loss at step 1000: 1508.5321044921875
Minibatch accuracy: 76.6
Validation accuracy: 80.2
Minibatch loss at step 1500: 1385.5096435546875
Minibatch accuracy: 78.1
Validation accuracy: 80.7
Minibatch loss at step 2000: 1318.0615234375
Minibatch accuracy: 81.2
Validation accuracy: 80.5
Minibatch loss at step 2500: 1295.7269287109375
Minibatch accuracy: 85.2
Validation accuracy: 80.7
Minibatch loss at step 3000: 1322.8468017578125
Minibatch accuracy: 78.1
Validation accuracy: 80.2
Test accuracy: 87.2
CPU times: user 22.4 s, sys: 3.56 s, total: 25.9 s
Wall time: 30.5 s


2 layer neural net without L2 regularization and with drop-out

In [18]:
nodes_layer_1 = 1024
nodes_layer_2 = 512
batch_size = 128
alpha = 0.0025

nn_graph = tf.Graph()
with nn_graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    """VARIABLES"""
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, nodes_layer_1]))
    biases_1  = tf.Variable(tf.zeros([nodes_layer_1]))
    
    weights_2 = tf.Variable(tf.truncated_normal([nodes_layer_1, nodes_layer_2]))
    biases_2  = tf.Variable(tf.zeros([nodes_layer_2]))
    
    weights_3 = tf.Variable(tf.truncated_normal([nodes_layer_2, num_labels]))
    biases_3  = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    # Dropout regularization for half of the nodes
    drop_nodes_1 = tf.nn.dropout(relu_layer_1, 0.5)
    
    logits_2 = tf.matmul(drop_nodes_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    drop_nodes_2 = tf.nn.dropout(relu_layer_2, 0.5)
    
    logits_3 = tf.matmul(drop_nodes_2, weights_3) + biases_3
    
    
    # Do not regularize biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_3, labels=tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(alpha).minimize(loss)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_3)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    valid_prediction = tf.nn.softmax(logits_3)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    test_prediction = tf.nn.softmax(logits_3)

In [19]:
%%time

num_steps = 3001

print('With dropout regularization')

with tf.Session(graph=nn_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare minibatch dict
        minibatch_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=minibatch_dict)
        
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

With dropout regularization
Initialized
Minibatch loss at step 0: 9798.580078125
Minibatch accuracy: 5.5
Validation accuracy: 16.8
Minibatch loss at step 500: 427.8526611328125
Minibatch accuracy: 72.7
Validation accuracy: 81.0
Minibatch loss at step 1000: 222.0453338623047
Minibatch accuracy: 63.3
Validation accuracy: 79.0
Minibatch loss at step 1500: 112.60509490966797
Minibatch accuracy: 59.4
Validation accuracy: 77.7
Minibatch loss at step 2000: 56.366905212402344
Minibatch accuracy: 67.2
Validation accuracy: 78.1
Minibatch loss at step 2500: 60.406036376953125
Minibatch accuracy: 66.4
Validation accuracy: 78.4
Minibatch loss at step 3000: 59.83869171142578
Minibatch accuracy: 58.6
Validation accuracy: 76.7
Test accuracy: 83.7
CPU times: user 22.4 s, sys: 3.12 s, total: 25.5 s
Wall time: 29.7 s


Add Xavier Initialization to drop out regularization  
Increase nodes and learning rate

In [20]:
nodes_layer_1 = 1024
nodes_layer_2 = 512
batch_size = 128
alpha = 0.05

nn_graph = tf.Graph()
with nn_graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    """VARIABLES"""
    weights_1 = tf.get_variable("weights_1", shape=[image_size * image_size, nodes_layer_1], initializer=tf.contrib.layers.xavier_initializer())
    biases_1  = tf.Variable(tf.zeros([nodes_layer_1]))
    
    weights_2 = tf.get_variable("weights_2", shape=[nodes_layer_1, nodes_layer_2], initializer=tf.contrib.layers.xavier_initializer())
    biases_2  = tf.Variable(tf.zeros([nodes_layer_2]))
    
    weights_3 = tf.get_variable("weights_3", shape=[nodes_layer_2, num_labels], initializer=tf.contrib.layers.xavier_initializer())
    biases_3  = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    # Dropout regularization for half of the nodes
    drop_nodes_1 = tf.nn.dropout(relu_layer_1, 0.5)
    
    logits_2 = tf.matmul(drop_nodes_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    drop_nodes_2 = tf.nn.dropout(relu_layer_2, 0.5)
    
    logits_3 = tf.matmul(drop_nodes_2, weights_3) + biases_3
    
    
    # Do not regularize biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_3, labels=tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(alpha).minimize(loss)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_3)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    valid_prediction = tf.nn.softmax(logits_3)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    test_prediction = tf.nn.softmax(logits_3)

In [21]:
%%time

num_steps = 3001

print('With dropout regularization')

with tf.Session(graph=nn_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare minibatch dict
        minibatch_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=minibatch_dict)
        
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

With dropout regularization
Initialized
Minibatch loss at step 0: 2.53611159324646
Minibatch accuracy: 8.6
Validation accuracy: 18.1
Minibatch loss at step 500: 0.6650431752204895
Minibatch accuracy: 79.7
Validation accuracy: 83.7
Minibatch loss at step 1000: 0.7383277416229248
Minibatch accuracy: 81.2
Validation accuracy: 84.5
Minibatch loss at step 1500: 0.6384693384170532
Minibatch accuracy: 78.9
Validation accuracy: 85.6
Minibatch loss at step 2000: 0.49255385994911194
Minibatch accuracy: 86.7
Validation accuracy: 85.8
Minibatch loss at step 2500: 0.40739357471466064
Minibatch accuracy: 90.6
Validation accuracy: 86.3
Minibatch loss at step 3000: 0.7124489545822144
Minibatch accuracy: 78.1
Validation accuracy: 86.5
Test accuracy: 92.4
CPU times: user 21.7 s, sys: 3.38 s, total: 25 s
Wall time: 29.7 s


Run with drop out and higher learning rate

In [22]:
nodes_layer_1 = 1024
nodes_layer_2 = 512
batch_size = 128
alpha = 0.08

nn_graph = tf.Graph()
with nn_graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    """VARIABLES"""
    weights_1 = tf.get_variable("weights_1", shape=[image_size * image_size, nodes_layer_1], initializer=tf.contrib.layers.xavier_initializer())
    biases_1  = tf.Variable(tf.zeros([nodes_layer_1]))
    
    weights_2 = tf.get_variable("weights_2", shape=[nodes_layer_1, nodes_layer_2], initializer=tf.contrib.layers.xavier_initializer())
    biases_2  = tf.Variable(tf.zeros([nodes_layer_2]))
    
    weights_3 = tf.get_variable("weights_3", shape=[nodes_layer_2, num_labels], initializer=tf.contrib.layers.xavier_initializer())
    biases_3  = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    # Dropout regularization for half of the nodes
    drop_nodes_1 = tf.nn.dropout(relu_layer_1, 0.5)
    
    logits_2 = tf.matmul(drop_nodes_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    drop_nodes_2 = tf.nn.dropout(relu_layer_2, 0.5)
    
    logits_3 = tf.matmul(drop_nodes_2, weights_3) + biases_3
    
    
    # Do not regularize biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_3, labels=tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(alpha).minimize(loss)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_3)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    valid_prediction = tf.nn.softmax(logits_3)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    test_prediction = tf.nn.softmax(logits_3)

In [23]:
%%time

num_steps = 3001

print('With dropout regularization')

with tf.Session(graph=nn_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare minibatch dict
        minibatch_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=minibatch_dict)
        
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

With dropout regularization
Initialized
Minibatch loss at step 0: 2.536372184753418
Minibatch accuracy: 8.6
Validation accuracy: 24.4
Minibatch loss at step 500: 0.6325914859771729
Minibatch accuracy: 81.2
Validation accuracy: 84.3
Minibatch loss at step 1000: 0.719950795173645
Minibatch accuracy: 82.0
Validation accuracy: 85.0
Minibatch loss at step 1500: 0.604725182056427
Minibatch accuracy: 79.7
Validation accuracy: 86.2
Minibatch loss at step 2000: 0.4676779508590698
Minibatch accuracy: 85.9
Validation accuracy: 86.5
Minibatch loss at step 2500: 0.3927905857563019
Minibatch accuracy: 90.6
Validation accuracy: 87.0
Minibatch loss at step 3000: 0.6994513869285583
Minibatch accuracy: 77.3
Validation accuracy: 87.3
Test accuracy: 93.1
CPU times: user 22.2 s, sys: 3.34 s, total: 25.5 s
Wall time: 29.4 s


Add decay of learning rate

In [24]:
nodes_layer_1 = 1024
nodes_layer_2 = 512
batch_size = 128
alpha = 0.12

nn_graph = tf.Graph()
with nn_graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    """VARIABLES"""
    # Needed to decay the learning rate
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(alpha, global_step, 100, 0.95, staircase=True)
    
    weights_1 = tf.get_variable("weights_1", shape=[image_size * image_size, nodes_layer_1], initializer=tf.contrib.layers.xavier_initializer())
    biases_1  = tf.Variable(tf.zeros([nodes_layer_1]))
    
    weights_2 = tf.get_variable("weights_2", shape=[nodes_layer_1, nodes_layer_2], initializer=tf.contrib.layers.xavier_initializer())
    biases_2  = tf.Variable(tf.zeros([nodes_layer_2]))
    
    weights_3 = tf.get_variable("weights_3", shape=[nodes_layer_2, num_labels], initializer=tf.contrib.layers.xavier_initializer())
    biases_3  = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    # Dropout regularization for half of the nodes
    drop_nodes_1 = tf.nn.dropout(relu_layer_1, 0.5)
    
    logits_2 = tf.matmul(drop_nodes_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    drop_nodes_2 = tf.nn.dropout(relu_layer_2, 0.5)
    
    logits_3 = tf.matmul(drop_nodes_2, weights_3) + biases_3
    
    
    # Do not regularize biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_3, labels=tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_3)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    valid_prediction = tf.nn.softmax(logits_3)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    test_prediction = tf.nn.softmax(logits_3)

In [25]:
%%time

num_steps = 3001

print('With dropout regularization')

with tf.Session(graph=nn_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare minibatch dict
        minibatch_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=minibatch_dict)
        
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

With dropout regularization
Initialized
Minibatch loss at step 0: 2.515817165374756
Minibatch accuracy: 5.5
Validation accuracy: 30.0
Minibatch loss at step 500: 0.5923751592636108
Minibatch accuracy: 83.6
Validation accuracy: 84.7
Minibatch loss at step 1000: 0.7116248607635498
Minibatch accuracy: 78.9
Validation accuracy: 85.7
Minibatch loss at step 1500: 0.5744303464889526
Minibatch accuracy: 81.2
Validation accuracy: 86.1
Minibatch loss at step 2000: 0.44304904341697693
Minibatch accuracy: 87.5
Validation accuracy: 86.5
Minibatch loss at step 2500: 0.42731305956840515
Minibatch accuracy: 88.3
Validation accuracy: 86.9
Minibatch loss at step 3000: 0.7177857756614685
Minibatch accuracy: 82.8
Validation accuracy: 87.1
Test accuracy: 92.7
CPU times: user 23.3 s, sys: 3.45 s, total: 26.7 s
Wall time: 30 s


Tune both the learning rate decay and the number of nodes

In [26]:
nodes_layer_1 = 2048
nodes_layer_2 = 1024
batch_size = 128
alpha = 0.16

nn_graph = tf.Graph()
with nn_graph.as_default():
    
    """RANDOM SEED"""
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    """VARIABLES"""
    # Needed to decay the learning rate
    global_step   = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(alpha, global_step, 100, 0.95, staircase=True)
    
    weights_1 = tf.get_variable("weights_1", shape=[image_size * image_size, nodes_layer_1], initializer=tf.contrib.layers.xavier_initializer())
    biases_1  = tf.Variable(tf.zeros([nodes_layer_1]))
    
    weights_2 = tf.get_variable("weights_2", shape=[nodes_layer_1, nodes_layer_2], initializer=tf.contrib.layers.xavier_initializer())
    biases_2  = tf.Variable(tf.zeros([nodes_layer_2]))
    
    weights_3 = tf.get_variable("weights_3", shape=[nodes_layer_2, num_labels], initializer=tf.contrib.layers.xavier_initializer())
    biases_3  = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    # Dropout regularization for half of the nodes
    drop_nodes_1 = tf.nn.dropout(relu_layer_1, 0.5)
    
    logits_2 = tf.matmul(drop_nodes_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    drop_nodes_2 = tf.nn.dropout(relu_layer_2, 0.5)
    
    logits_3 = tf.matmul(drop_nodes_2, weights_3) + biases_3
    
    
    # Do not regularize biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_3, labels=tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_3)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    valid_prediction = tf.nn.softmax(logits_3)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer_1 = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer_1, weights_2) + biases_2
    relu_layer_2 = tf.nn.relu(logits_2)
    
    logits_3 = tf.matmul(relu_layer_2, weights_3) + biases_3
       
    test_prediction = tf.nn.softmax(logits_3)

In [27]:
%%time

num_steps = 3001

print('Training with dropout regularization')
print('Nodes of size: ' + str(nodes_layer_1) + ' and ' + str(nodes_layer_2))
print('Initial learning rate alpha: ' + str(alpha))

with tf.Session(graph=nn_graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare minibatch dict
        minibatch_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=minibatch_dict)
        
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy:  {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
            print('Learning rate now: ' + str(learning_rate.eval()))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

Training with dropout regularization
Nodes of size: 2048 and 1024
Initial learning rate alpha: 0.16
Initialized
Minibatch loss at step 0: 2.4204211235046387
Minibatch accuracy:  10.9
Validation accuracy: 38.5
Learning rate now: 0.16
Minibatch loss at step 500: 0.5154994130134583
Minibatch accuracy:  83.6
Validation accuracy: 85.4
Learning rate now: 0.123805
Minibatch loss at step 1000: 0.6738272905349731
Minibatch accuracy:  81.2
Validation accuracy: 86.3
Learning rate now: 0.0957979
Minibatch loss at step 1500: 0.5616711378097534
Minibatch accuracy:  83.6
Validation accuracy: 87.0
Learning rate now: 0.0741266
Minibatch loss at step 2000: 0.39494287967681885
Minibatch accuracy:  88.3
Validation accuracy: 87.5
Learning rate now: 0.0573577
Minibatch loss at step 2500: 0.3192635178565979
Minibatch accuracy:  91.4
Validation accuracy: 87.7
Learning rate now: 0.0443823
Minibatch loss at step 3000: 0.6535000205039978
Minibatch accuracy:  81.2
Validation accuracy: 88.0
Learning rate now: 0.03